In [1]:
# 캘리 실행 후 파일 저장

In [2]:
import numpy as np
import cv2
import glob
import os
from tqdm import tqdm

In [3]:
CHECKERBOARD = (9, 6)       # 체스보드 코너 수
SQUARE_SIZE = 25            # 한 칸의 실제 크기 (mm) - 자로 잰 실제 값 입력 권장
IMAGE_DIR = "cal" # 이미지가 저장된 폴더

print(f"이미지 폴더 '{IMAGE_DIR}'에서 데이터 로딩 중...")

이미지 폴더 'cal'에서 데이터 로딩 중...


In [4]:
# 3D 포인트 준비 (0,0,0), (1,0,0), ...
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp = objp * SQUARE_SIZE

# 배열 저장용 리스트
objpoints = [] # 3D points
imgpointsL = [] # 2D points (Left)
imgpointsR = [] # 2D points (Right)

# 이미지 파일 목록 불러오기 (정렬 중요)
images_left = sorted(glob.glob(f"{IMAGE_DIR}/left/l_*.jpg"))
images_right = sorted(glob.glob(f"{IMAGE_DIR}/right/r_*.jpg"))

print(f"발견된 이미지 파일: {len(images_left)}")
print(f"발견된 이미지 파일: {len(images_right)}")

if len(images_left) and len(images_right) == 0:
    print("이미지가 없습니다. 경로를 확인하세요.")
    exit()

발견된 이미지 파일: 120
발견된 이미지 파일: 120


In [5]:
img_shape = None
valid_count = 0

# 이미지 하나씩 정밀 분석
for i, (img_l_path, img_r_path) in enumerate(tqdm(zip(images_left, images_right), unit=" image")):
    imgL = cv2.imread(img_l_path)
    imgR = cv2.imread(img_r_path)

    if img_shape is None:
        img_shape = imgL.shape[:2][::-1]

    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    # 코너 찾기
    retL, cornersL = cv2.findChessboardCorners(grayL, CHECKERBOARD, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, CHECKERBOARD, None)

    if retL and retR:
        objpoints.append(objp)

        # 코너 위치 정밀 보정 (Subpixel)
        criteria_subpix = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        cornersL_opt = cv2.cornerSubPix(grayL, cornersL, (11, 11), (-1, -1), criteria_subpix)
        cornersR_opt = cv2.cornerSubPix(grayR, cornersR, (11, 11), (-1, -1), criteria_subpix)

        imgpointsL.append(cornersL_opt)
        imgpointsR.append(cornersR_opt)
        valid_count += 1
        
        # # 진행률 표시
        # if i % 10 == 0:
        #     print(f"처리 중... {i}/{len(images_left)}")
    else:
        print(f"Skip: {os.path.basename(img_l_path)} (인식 실패)")

print(f"\n유효한 데이터: {valid_count}")

120 image [00:05, 21.72 image/s]


유효한 데이터: 120


In [6]:
print(len(objpoints))
print(len(imgpointsL))
print(len(imgpointsR))

120
120
120


In [7]:
# 1. 개별 카메라 캘리브레이션
print("왼쪽 카메라 캘리 중...")
retL, mtxL, distL, _, _ = cv2.calibrateCamera(objpoints, imgpointsL, img_shape, None, None)
print("오른쪽 카메라 캘리 중...")
retR, mtxR, distR, _, _ = cv2.calibrateCamera(objpoints, imgpointsR, img_shape, None, None)

# 2. 스테레오 캘리브레이션
print("캘리 최적화 중...")
flags = cv2.CALIB_FIX_INTRINSIC
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

retStereo, newMtxL, distL, newMtxR, distR, R, T, E, F = cv2.stereoCalibrate(
    objpoints, imgpointsL, imgpointsR,
    mtxL, distL, mtxR, distR,
    img_shape, criteria=criteria, flags=flags)

print(f"----------------------------------------")
print(f"★ 캘리브레이션 완료! RMS 오차: {retStereo:.4f}")
# print(f"(보통 0.5 이하면 좋음, 120장이면 1.0 근처여도 훌륭함)")
print(f"----------------------------------------")

# 3. 저장
np.savez("stereo_calib.npz", 
         mtxL=newMtxL, distL=distL, 
         mtxR=newMtxR, distR=distR, 
         R=R, T=T)

print("파일 저장 완료: stereo_calib.npz")

왼쪽 카메라 캘리 중...
오른쪽 카메라 캘리 중...
캘리 최적화 중...
----------------------------------------
★ 캘리브레이션 완료! RMS 오차: 1.0416
----------------------------------------
파일 저장 완료: stereo_calib.npz
